In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils.engine as ue
import utils.engine0d as ue0d
import utils.visualizer as uv

In [ ]:
%reload_ext autoreload
%reload_ext line_profiler
%autoreload 2

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
def make_linear_profile_ice(h_ice, T_ib, T_is, N_cells):
    dz_cells = np.ones(N_cells)*(h_ice/N_cells)
    T_cells = T_ib + np.arange(0.5, N_cells)/N_cells * (T_is - T_ib)
    return dz_cells, T_ib, T_cells, T_is

In [ ]:
make_linear_profile_ice(2.0, 0.0, -10.0, 10)

# 1.Тестирование функции ice_freezing_0d (и сравнение с ice_freezing)

In [ ]:
def test_ice_freezing_0d(N, # количество ячеек в сетке вывода
                         N_time_steps,
                         N_pseudoiter,
                         time_step,
                         thickness_init,
                         T_a,
                         T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_ia_prev, thickness_prev = 0.0, 0.0
    T_ia_new, thickness_new = T_a(0.0)/2.0 , thickness_init
    
    dz_cells_ice_init, T_ib_init, T_cells_init, T_is_init = make_linear_profile_ice(h_ice=thickness_init,
                                                                                    T_ib=T_o(0.0),
                                                                                    T_is=T_ia_new,
                                                                                    N_cells=N)
    
    process = ue.Process([dz_cells_ice_init], [np.zeros(10)],
                         [0.0],
                         [T_ib_init], [T_cells_init], [T_is_init], [np.zeros(10)*np.nan], [np.nan],
                         [[ue.rho_i]*N],
                         [False])
    
   
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_ia_prev, thickness_prev = T_ia_new, thickness_new 
        
        T_ia_new, thickness_new = \
        ue0d.ice_freezing_0d(Toi=T_ib_init,
                             Tia_init=T_ia_prev,
                             F_atm=F_atm,
                             F_ocn=F_ocn,
                             thickness_i_init=thickness_prev,
                             N_pseudoiter=N_pseudoiter,
                             time_step=time_step)
        
        # вывод
        dz_cells_ice, T_ib, T_cells, T_is = make_linear_profile_ice(h_ice=thickness_new,
                                                                    T_ib=T_o(time),
                                                                    T_is=T_ia_new,
                                                                    N_cells=N)
        
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [np.zeros(10)], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_cells.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_is)
        process.snow_temp_history = np.append(process.snow_temp_history, [np.zeros(10)*np.nan], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, np.nan)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, False)
        
        print('time = ', time)
    
    return process

In [ ]:
def test_ice_freezing_1d(N,
                         N_time_steps,
                         N_pseudoiter,
                         time_step,
                         thickness_init,
                         T_a,
                         T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_ia_new = T_a(0.0)/2.0
    dz_cells_ice_init, T_ib_init, T_cells_init, T_is_init = make_linear_profile_ice(h_ice=thickness_init,
                                                                                    T_ib=T_o(0.0),
                                                                                    T_is=T_ia_new,
                                                                                    N_cells=N)
    
    T_ia_prev, dz_cells_ice_prev =  T_is_init, dz_cells_ice_init
    dz_cells_ice_new = dz_cells_ice_init
    T_cells_prev , T_cells_new = T_cells_init, T_cells_init
    
    process = ue.Process([dz_cells_ice_init], [np.zeros(10)],
                         [0.0],
                         [T_ib_init], [T_cells_init], [T_is_init], [np.zeros(10)*np.nan], [np.nan],
                         [[ue.rho_i]*N],
                         [False])
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_ia_prev, dz_cells_ice_prev, T_cells_prev = T_ia_new, dz_cells_ice_new, T_cells_new 
        
        T_cells_new, T_ia_new, dz_cells_ice_new = \
        ue.ice_freezing(Toi=T_ib_init,
                        Ti=T_cells_prev,
                        Tia=T_ia_prev,
                        F_atm=F_atm,
                        F_ocn=F_ocn,
                        F_sw=0.0,
                        dzi=dz_cells_ice_prev,
                        salinity=make_linear_profile_ice(1.0, 4.0, 1.0, N)[2],
                        N_pseudoiter=N_pseudoiter,
                        time_step=time_step,
                        tol=1e-12)
        
        # вывод
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice_new.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [np.zeros(10)], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_cells_new.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_ia_new)
        process.snow_temp_history = np.append(process.snow_temp_history, [np.zeros(10)*np.nan], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, np.nan)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, False)
        
        print('time = ', time)
    
    return process

In [ ]:
# расчет тестов
test_ice_freezing_process_0d = test_ice_freezing_0d(N=10,
                                                    N_time_steps=500,
                                                    N_pseudoiter=50,
                                                    time_step=3600.0,
                                                    thickness_init=1.0,
                                                    T_a=lambda time: -20.0,
                                                    T_o=lambda time: ue.Tf_i(30.0))

test_ice_freezing_process_1d = test_ice_freezing_1d(N=10,
                                                    N_time_steps=500,
                                                    N_pseudoiter=50,
                                                    time_step=3600.0,
                                                    thickness_init=1.0,
                                                    T_a=lambda time: -20.0,
                                                    T_o=lambda time: ue.Tf_i(30.0))

In [ ]:
# анимация
uv.animate([test_ice_freezing_process_0d, test_ice_freezing_process_1d], names=['0D', '1D'])

In [ ]:
# временной ряд разности
uv.timeseries_err(process_sim=test_ice_freezing_process_1d,
                  process_data=test_ice_freezing_process_0d,
                  tmin_err=-4., tmax_err=4., step_err=.2,
                  figsize=(20, 10))

# 2.Тестирование функции ice_melting_0d (и сравнение с ice_melting)

In [ ]:
def test_ice_melting_0d(N,
                        N_time_steps,
                        N_pseudoiter,
                        time_step,
                        thickness_init,
                        T_a,
                        T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_ia_prev, thickness_prev = 0.0, 0.0
    T_ia_new, thickness_new = T_a(0.0)/2.0 , thickness_init
    
    dz_cells_ice_init, T_ib_init, T_cells_init, T_is_init = make_linear_profile_ice(h_ice=thickness_init,
                                                                                    T_ib=T_o(0.0),
                                                                                    T_is=T_ia_new,
                                                                                    N_cells=N)
    
    process = ue.Process([dz_cells_ice_init], [np.zeros(10)],
                         [0.0],
                         [T_ib_init], [T_cells_init], [T_is_init], [np.zeros(10)*np.nan], [np.nan],
                         [[ue.rho_i]*N],
                         [False])
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_ia_prev, thickness_prev = T_ia_new, thickness_new
        
        T_ia_new, thickness_new = \
        ue0d.ice_freezing_0d(Toi=T_ib_init,
                             Tia_init=T_ia_prev,
                             F_atm=F_atm,
                             F_ocn=F_ocn,
                             thickness_i_init=thickness_prev,
                             N_pseudoiter=N_pseudoiter,
                             time_step=time_step)
        
        if (T_ia_new >= ue.Tf_i(1.0)):
            
            T_ia_new = ue.Tf_i(1.0)
            
            thickness_new = \
            ue0d.ice_melting_0d(Toi=T_ib_init,
                                F_atm=F_atm,
                                F_ocn=F_ocn,
                                thickness_i_init=thickness_prev,
                                N_pseudoiter=N_pseudoiter,
                                time_step=time_step)
            
            print('ice melting')
            
        else:
            print('ice freezing')
        
        # вывод
        
        dz_cells_ice, T_ib, T_cells, T_is = make_linear_profile_ice(h_ice=thickness_new,
                                                                    T_ib=T_o(time),
                                                                    T_is=T_ia_new,
                                                                    N_cells=N)
        
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [np.zeros(10)], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_cells.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_is)
        process.snow_temp_history = np.append(process.snow_temp_history, [np.zeros(10)*np.nan], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, np.nan)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, False)
        
        print('time = ', time)
    
    return process

In [ ]:
def test_ice_melting_1d(N,
                        N_time_steps,
                        N_pseudoiter,
                        time_step,
                        thickness_init,
                        T_a,
                        T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_ia_new = T_a(0.0)/2.0
    dz_cells_ice_init, T_ib_init, T_cells_init, T_is_init = make_linear_profile_ice(h_ice=thickness_init,
                                                                                    T_ib=T_o(0.0),
                                                                                    T_is=T_ia_new,
                                                                                    N_cells=N)
    
    T_ia_prev, dz_cells_ice_prev =  T_is_init, dz_cells_ice_init
    dz_cells_ice_new = dz_cells_ice_init
    T_cells_prev , T_cells_new = T_cells_init, T_cells_init
    
    process = ue.Process([dz_cells_ice_init], [np.zeros(10)],
                         [0.0],
                         [T_ib_init], [T_cells_init], [T_is_init], [np.zeros(10)*np.nan], [np.nan],
                         [[ue.rho_i]*N],
                         [False])
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_ia_prev, dz_cells_ice_prev, T_cells_prev = T_ia_new, dz_cells_ice_new, T_cells_new 
        
        T_cells_new, T_ia_new, dz_cells_ice_new = \
        ue.ice_freezing(Toi=T_ib_init,
                        Ti=T_cells_prev,
                        Tia=T_ia_prev,
                        F_atm=F_atm,
                        F_ocn=F_ocn,
                        F_sw=0.0,
                        dzi=dz_cells_ice_prev,
                        salinity=make_linear_profile_ice(1.0, 4.0, 1.0, N)[2],
                        N_pseudoiter=N_pseudoiter,
                        time_step=time_step,
                        tol=1e-12)
        
        if (T_ia_new >= ue.Tf_i(1.0)):
            
            T_ia_new = ue.Tf_i(1.0)
            
            T_cells_new, dz_cells_ice_new = \
            ue.ice_melting(Toi=T_ib_init,
                           Ti=T_cells_prev,
                           Tia_old=T_ia_prev,
                           F_atm=F_atm,
                           F_ocn=F_ocn,
                           F_sw=0.0,
                           dzi=dz_cells_ice_prev,
                           salinity=make_linear_profile_ice(1.0, 4.0, 1.0, N)[2],
                           N_pseudoiter=N_pseudoiter,
                           time_step=time_step,
                           tol=1e-12)
            
            print('ice melting')
            
        else:
            print('ice freezing')
        
        # вывод
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice_new.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [np.zeros(10)], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_cells_new.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_ia_new)
        process.snow_temp_history = np.append(process.snow_temp_history, [np.zeros(10)*np.nan], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, np.nan)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, False)
        
        print('time = ', time)
    
    return process

In [ ]:
def warming_atmosphere(time, total_time, low_temp, high_temp):
    if (time <= total_time/2.0):
        return low_temp + (high_temp - low_temp)/(0.5*total_time)*time
    return high_temp

In [ ]:
plt.figure(figsize=(20, 10))
T_interval = np.arange(500)*3600
plt.plot(T_interval, [warming_atmosphere(T, 500*3600, -20.0, 10.0) for T in T_interval])
plt.grid()
plt.show()

In [ ]:
test_ice_melting_process_0d = test_ice_melting_0d(N=10,
                                                  N_time_steps=500,
                                                  N_pseudoiter=50,
                                                  time_step=3600.0,
                                                  thickness_init=1.0,
                                                  T_a=lambda time: warming_atmosphere(time, 500*3600.0, -20.0, 5.0),
                                                  T_o=lambda time: ue.Tf_i(30.0))

test_ice_melting_process_1d = test_ice_melting_1d(N=10,
                                                  N_time_steps=500,
                                                  N_pseudoiter=50,
                                                  time_step=3600.0,
                                                  thickness_init=1.0,
                                                  T_a=lambda time: warming_atmosphere(time, 500*3600.0, -20.0, 5.0),
                                                  T_o=lambda time: ue.Tf_i(30.0))

In [ ]:
uv.animate([test_ice_melting_process_0d, test_ice_melting_process_1d], names=['0D','1D'])

In [ ]:
# временной ряд разности
uv.timeseries_err(process_sim=test_ice_melting_process_1d,
                  process_data=test_ice_melting_process_0d,
                  tmin_err=-4., tmax_err=4., step_err=.2,
                  figsize=(20, 10))

# 3.Тестирование функции snow_ice_freezing_0d (и сравнение с snow_ice_freezing)

In [ ]:
def test_snow_ice_freezing_0d(N,
                              N_time_steps,
                              N_pseudoiter,
                              time_step,
                              ice_thickness_init,
                              snow_thickness_init,
                              T_a,
                              p,
                              T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_sa_new = T_a(0.0)/2.0
    T_is_new = T_a(0.0)/5.0
    dz_cells_ice_init, T_ib_init, T_ice_cells_init, T_is_init = make_linear_profile_ice(h_ice=ice_thickness_init,
                                                                                        T_ib=T_o(0.0),
                                                                                        T_is=T_is_new,
                                                                                        N_cells=N)
    
    dz_cells_snow_init, T_is_init, T_snow_cells_init, T_sa_init = make_linear_profile_ice(h_ice=snow_thickness_init,
                                                                                          T_ib=T_is_new,
                                                                                          T_is=T_sa_new,
                                                                                          N_cells=10)
    
    T_is_prev, T_sa_prev =  T_is_new, T_sa_new
    thickness_ice_prev, thickness_snow_prev = ice_thickness_init, snow_thickness_init
    thickness_ice_new, thickness_snow_new = ice_thickness_init, snow_thickness_init
    
    process = ue.Process([dz_cells_ice_init], [dz_cells_snow_init],
                         [0.0],
                         [T_ib_init], [T_ice_cells_init], [T_is_init], [T_snow_cells_init], [T_sa_init],
                         [[ue.rho_i]*N],
                         [True])
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_is_prev, T_sa_prev = T_is_new, T_sa_new
        thickness_ice_prev, thickness_snow_prev = thickness_ice_new, thickness_snow_new
        
        T_is_new, T_sa_new, thickness_ice_new, thickness_snow_new = \
        ue0d.snow_ice_freezing_0d(Toi=T_ib_init,
                                  Tis_init=T_is_prev,
                                  Tsa_init=T_sa_prev,
                                  Ta=T_a(time),
                                  F_atm=F_atm,
                                  F_ocn=F_ocn,
                                  ice_thickness_init=thickness_ice_prev,
                                  snow_thickness_init=thickness_snow_prev,
                                  N_pseudoiter=N_pseudoiter,
                                  time_step=time_step,
                                  p=p(time))
        
        # вывод
        dz_cells_ice, T_ib, T_ice_cells, T_is = make_linear_profile_ice(h_ice=thickness_ice_new,
                                                                        T_ib=T_o(0.0),
                                                                        T_is=T_is_new,
                                                                        N_cells=N)
    
        dz_cells_snow, T_is_init, T_snow_cells, T_sa = make_linear_profile_ice(h_ice=thickness_snow_new,
                                                                               T_ib=T_is_new,
                                                                               T_is=T_sa_new,
                                                                               N_cells=10)
        
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [dz_cells_snow.copy()], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_ice_cells.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_is)
        process.snow_temp_history = np.append(process.snow_temp_history, [T_snow_cells.copy()], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, T_sa)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, True)
        
        print('time = ', time)
    
    return process

In [ ]:
def test_snow_ice_freezing_1d(N,
                              N_time_steps,
                              N_pseudoiter,
                              time_step,
                              ice_thickness_init,
                              snow_thickness_init,
                              T_a,
                              p,
                              T_o):
    
    # форсинг
    time = 0.0
    F_ocn = lambda T: 0.0
    F_atm = lambda T: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(T_a(time) - T)
    
    # инициализация процесса
    T_sa_new = T_a(0.0)/2.0
    T_is_new = T_a(0.0)/5.0
    dz_cells_ice_init, T_ib_init, T_ice_cells_init, T_is_init = make_linear_profile_ice(h_ice=ice_thickness_init,
                                                                                        T_ib=T_o(0.0),
                                                                                        T_is=T_is_new,
                                                                                        N_cells=N)
    
    dz_cells_snow_init, T_is_init, T_snow_cells_init, T_sa_init = make_linear_profile_ice(h_ice=snow_thickness_init,
                                                                                          T_ib=T_is_new,
                                                                                          T_is=T_sa_new,
                                                                                          N_cells=10)
    
    T_is_prev, T_sa_prev =  T_is_new, T_sa_new
    dz_cells_ice_new, dz_cells_snow_new = dz_cells_ice_init, dz_cells_snow_init
    T_ice_cells_prev, T_snow_cells_prev = T_ice_cells_init, T_snow_cells_init
    T_ice_cells_new, T_snow_cells_new = T_ice_cells_init, T_snow_cells_init
    
    process = ue.Process([dz_cells_ice_init], [dz_cells_snow_init],
                         [0.0],
                         [T_ib_init], [T_ice_cells_init], [T_is_init], [T_snow_cells_init], [T_sa_init],
                         [[ue.rho_i]*N],
                         [True])
    
    # временной цикл
    for it in range(N_time_steps):
        
        time += time_step
        
        T_is_prev, T_sa_prev = T_is_new, T_sa_new
        T_ice_cells_prev, T_snow_cells_prev = T_ice_cells_new, T_snow_cells_new
        dz_cells_ice_prev, dz_cells_snow_prev = dz_cells_ice_new, dz_cells_snow_new
        
        T_ice_cells_new, T_snow_cells_new, T_is_new, T_sa_new, dz_cells_ice_new, dz_cells_snow_new = \
        ue.snow_ice_freezing(Toi=T_ib_init,
                             Ti=T_ice_cells_prev,
                             Ts=T_snow_cells_prev,
                             Tis=T_is_prev,
                             Tsa=T_sa_prev,
                             Ta=T_a(time),
                             F_atm=F_atm,
                             F_ocn=F_ocn,
                             F_sw=0.0,
                             dzi=dz_cells_ice_prev,
                             dzs=dz_cells_snow_prev,
                             salinity=make_linear_profile_ice(1.0, 4.0, 1.0, N)[2],
                             N_pseudoiter=N_pseudoiter,
                             time_step=time_step,
                             p=p(time),
                             tol=1e-12)
        
        # вывод
        process.ice_dz_history = np.append(process.ice_dz_history, [dz_cells_ice_new.copy()], axis=0)
        process.snow_dz_history = np.append(process.snow_dz_history, [dz_cells_snow_new.copy()], axis=0)
        process.timeline = np.append(process.timeline, time)
        process.oi_temp_history = np.append(process.oi_temp_history, T_o(time))
        process.ice_temp_history = np.append(process.ice_temp_history, [T_ice_cells_new.copy()], axis=0)
        process.is_temp_history = np.append(process.is_temp_history, T_is_new)
        process.snow_temp_history = np.append(process.snow_temp_history, [T_snow_cells_new.copy()], axis=0)
        process.sa_temp_history = np.append(process.sa_temp_history, T_sa_new)
        process.ice_density_history = np.append(process.ice_density_history, [[ue.rho_i]*N], axis=0)
        process.snow_presence_history = np.append(process.snow_presence_history, True)
        
        print('time = ', time)
    
    return process

In [ ]:
test_snow_ice_freezing_process_0d = test_snow_ice_freezing_0d(N=10,
                                                              N_time_steps=500,
                                                              N_pseudoiter=50,
                                                              time_step=3600.0,
                                                              ice_thickness_init=1.0,
                                                              snow_thickness_init=0.2,
                                                              T_a=lambda time: -20.0,
                                                              p=lambda time: 1e-8,
                                                              T_o=lambda time: ue.Tf_i(30.0))

test_snow_ice_freezing_process_1d = test_snow_ice_freezing_1d(N=10,
                                                              N_time_steps=500,
                                                              N_pseudoiter=50,
                                                              time_step=3600.0,
                                                              ice_thickness_init=1.0,
                                                              snow_thickness_init=0.2,
                                                              T_a=lambda time: -20.0,
                                                              p=lambda time: 1e-8,
                                                              T_o=lambda time: ue.Tf_i(30.0))

In [ ]:
uv.animate([test_snow_ice_freezing_process_0d, test_snow_ice_freezing_process_1d], names=['0D','1D'])

In [ ]:
# временной ряд разности
uv.timeseries_err(process_sim=test_snow_ice_freezing_process_1d,
                  process_data=test_snow_ice_freezing_process_0d,
                  tmin_err=-4., tmax_err=4., step_err=.2,
                  figsize=(20, 10))

# 4.Тестирование функции snow_melting_0d (и сравнение с snow_melting). На примере круглогодичной эволюции.

In [ ]:
def linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                    lowest_temp, highest_temp,
                    lowest_p, highest_p,
                    t):
    t %= (winter_time + spring_time + summer_time + autumn_time)
    if t < winter_time: # winter
        return lowest_temp, highest_p
    elif t < (winter_time + spring_time): # spring
        return lowest_temp + (t - winter_time)*(highest_temp - lowest_temp)/spring_time,\
               highest_p + (t - winter_time)*(lowest_p - highest_p)/spring_time
    elif t < (winter_time + spring_time + summer_time): # summer
        return highest_temp, lowest_p
    else: # autumn
        return highest_temp + (t - (winter_time + spring_time + summer_time))\
                             *(lowest_temp - highest_temp)/autumn_time,\
               lowest_p + (t - (winter_time + spring_time + summer_time))*(highest_p - lowest_p)/autumn_time

In [ ]:
winter_time = 350*3600.0
spring_time = 200*3600.0
summer_time = 150*3600.0
autumn_time = 200*3600.0
year_time = winter_time + spring_time + summer_time + autumn_time

lowest_temp = -30.0
highest_temp = 10.0
lowest_p = 0.0
highest_p = 1e-8

Tis_init = -15.0
Tsa_init = -20.0
ice_thickness_init = 4.0
snow_thickness_init = 0.1
Ni = 30
Ns = 5
dzi_init = np.full(Ni, ice_thickness_init/Ni)
dzs_init = np.full(Ns, snow_thickness_init/Ns)
salinity_init = np.linspace(4.0, 1.0, Ni)

In [ ]:
time_arr = np.linspace(0, 3*year_time, 300)

fig, ax1 = plt.subplots(figsize=(15, 10))
# ax1.set_title('Temperature and precipitation evolution', size=25)
ax2 = ax1.twinx()
ax1.plot(time_arr/3600.0,
         [linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                          lowest_temp, highest_temp,
                          lowest_p, highest_p, t)[0] for t in time_arr],
         label='tempreature', color='b', lw=3)
ax2.plot(time_arr/3600.0,
         [linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                          lowest_temp, highest_temp,
                          lowest_p, highest_p, t)[1] for t in time_arr],
         label='precipitation', color='r', lw=3)
ax1.spines['bottom'].set_color('b')
ax1.set_xlabel('Time, h', size=20)
ax1.set_ylabel(r'Air temperature, $^{\circ}$C', size=20)
ax2.set_ylabel(r'Precipitation rate, m s$^{-1}$', size=20)
ax1.tick_params(axis='both', labelsize=15)
ax2.tick_params(axis='y', labelsize=15)
ax2.spines['left'].set_color('b')
ax1.tick_params(axis='y', colors='b')
ax1.yaxis.label.set_color('b')
ax2.spines['right'].set_color('red')
ax2.tick_params(axis='y', colors='red')
ax2.yaxis.label.set_color('red')
ax2.yaxis.offsetText.set_fontsize(15)
ax1.grid(ls='--')
fig.tight_layout()
fig.savefig('media/prec_temp.png', dpi=200)
plt.show()

In [ ]:
def multiyear_evolution_0d(time_step,
                           N_pseudoiter=50,
                           n_years=1):
    
    # форсинг
    winter_time = 350*3600.0
    spring_time = 200*3600.0
    summer_time = 150*3600.0
    autumn_time = 200*3600.0
    year_time = winter_time + spring_time + summer_time + autumn_time
    
    lowest_temp = -30.0
    highest_temp = 10.0
    lowest_p = 0.0
    highest_p = 1e-7
    
    Ta = lambda time: linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                                      lowest_temp, highest_temp,
                                      lowest_p, highest_p, time)[0]
    
    p = lambda time: linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                                     lowest_temp, highest_temp,
                                     lowest_p, highest_p, time)[1]
    
    Toi = lambda time: ue.Tf_i(30.0)
    
    F_ocn = lambda T, time: 0.0
    F_atm = lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T)
    
    # инициализация процесса
    Tis_init = -15.0
    Tsa_init = -20.0
    
    ice_thickness_init = 4.0
    snow_thickness_init = 0.1
    
    process = ue0d.main_process_0d(time_step=time_step,
                                   time_end=year_time*n_years,
                                   N_pseudoiter=N_pseudoiter,
                                   Tis_init=Tis_init,
                                   Tsa_init=Tsa_init,
                                   ice_thickness_init=ice_thickness_init,
                                   snow_thickness_init=snow_thickness_init,
                                   snow_thickness_threshold=0.05,
                                   Toi=Toi,
                                   Ta=Ta,
                                   p=p,
                                   F_atm_ice=F_atm,
                                   F_atm_snow=F_atm,
                                   F_ocn=F_ocn)
    
    return process

In [ ]:
def multiyear_evolution_1d(N_cells_ice,
                           N_cells_snow,
                           time_step,
                           N_pseudoiter=50,
                           n_years=1):
    
    # форсинг
    winter_time = 350*3600.0
    spring_time = 200*3600.0
    summer_time = 150*3600.0
    autumn_time = 200*3600.0
    year_time = winter_time + spring_time + summer_time + autumn_time
    
    lowest_temp = -30.0
    highest_temp = 10.0
    lowest_p = 0.0
    highest_p = 1e-7
    
    Ta = lambda time: linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                                      lowest_temp, highest_temp,
                                      lowest_p, highest_p, time)[0]
    
    p = lambda time: linear_year_exp(winter_time, spring_time, summer_time, autumn_time,
                                     lowest_temp, highest_temp,
                                     lowest_p, highest_p, time)[1]
    
    Toi = lambda time: ue.Tf_i(30.0)
    
    F_ocn = lambda T, time: 0.0
    F_atm = lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*15.0*(Ta(time) - T)
    
    # инициализация процесса
    Tis_init = -15.0
    Tsa_init = -20.0
    
    ice_thickness_init = 4.0
    snow_thickness_init = 0.1
    
    dzi_init = np.full(N_cells_ice, ice_thickness_init/N_cells_ice)
    dzs_init = np.full(N_cells_snow, snow_thickness_init/N_cells_snow)
    salinity_init = np.linspace(4.0, 1.0, N_cells_ice)
    Ti_init = Toi(0.0) + np.arange(0.5, N_cells_ice)/N_cells_ice * (Tis_init - Toi(0.0))
    Ts_init = Tis_init + np.arange(0.5, N_cells_snow)/N_cells_snow * (Tsa_init - Tis_init)
    
    process = ue.main_process(time_step=time_step,
                              time_end=year_time*n_years,
                              N_pseudoiter=N_pseudoiter,
                              Ti_init=Ti_init,
                              Ts_init=Ts_init,
                              Tis_init=Tis_init,
                              Tsa_init=Tsa_init,
                              dzi_init=dzi_init,
                              dzs_init=dzs_init,
                              salinity=salinity_init,
                              snow_thickness_threshold=0.05,
                              Toi=Toi,
                              Ta=Ta,
                              p=p,
                              F_atm_ice=F_atm,
                              F_atm_snow=F_atm,
                              F_sw=lambda time: 0.0,
                              F_ocn=F_ocn)
    
    return process

In [ ]:
# расчет моделей
test_process_0d = multiyear_evolution_0d(time_step=3600.0,
                                         n_years=3)

test_process_1d = multiyear_evolution_1d(N_cells_ice=10,
                                         N_cells_snow=5,
                                         time_step=3600.0,
                                         n_years=3)

In [ ]:
uv.animate([test_process_0d, test_process_1d], names=['0D','1D'])

In [ ]:
# test_process_0d.timeline /= 3600*24
# test_process_1d.timeline /= 3600*24

In [ ]:
uv.timeseries_err(process_sim=test_process_1d,
                  process_data=test_process_0d,
                  label_sim='1-D', label_data='0-D', legend_loc='lower right',
                  tmin_err=-15., tmax_err=15., step_err=1.0,
                  levels_border = [-10, -5, -2, -1, 1, 2, 5, 10],
                  figsize=(20, 10), savepath='media/model_temp_diff.png')

In [ ]:
test_process_0d.timeline /= 3600*24

In [ ]:
uv.timeseries_img(test_process_0d, y_points=500, figsize=(20, 10),
                  cmap_ice='RdBu_r', tmin_ice=-23, tmax_ice=0, step_ice=2.0,
                  cmap_snow='PuOr_r', tmin_snow=-28, tmax_snow=0, step_snow=2.0, 
                  color_waterline='green', savepath='media/model_0d.png')

In [ ]:
uv.timeseries_img(test_process_1d, y_points=500, figsize=(20, 10),
                  cmap_ice='RdBu_r', tmin_ice=-23, tmax_ice=0, step_ice=2.0,
                  cmap_snow='PuOr_r', tmin_snow=-28, tmax_snow=0, step_snow=2.0, 
                  color_waterline='green', savepath='media/model_1d.png')

In [ ]:
fig, ax_Tsu = plt.subplots(figsize=(20, 15))

for process, label, alpha in zip([test_process_0d, test_process_1d], ['0-D', '1-D'], [1.0, 0.7]):
    ax_Tsu.plot(process.timeline*24,
                np.where(process.snow_presence_history, process.sa_temp_history, process.is_temp_history),
                label=label, lw=4, alpha=alpha)
# ax_ice_th.set_title('Ice thickness', size=25)
ax_Tsu.set_ylabel(r'Surface temperature, $^o$C', size=25)
ax_Tsu.set_xlabel('Time, h.', size=25)
ax_Tsu.tick_params(labelsize=20)
ax_Tsu.legend(prop={'size': 20})
ax_Tsu.grid(ls='--')
fig.savefig('media/model_surf_temps.png', bbox_inches='tight' ,dpi=200)